In [17]:
using Pkg
Pkg.activate(".")
Pkg.resolve()
Pkg.instantiate()

  Activating project at `/global/u2/j/jaimerz/PhD/MicroCanonicalHMC.jl/examples`
  No Changes to `/global/u2/j/jaimerz/PhD/MicroCanonicalHMC.jl/examples/Project.toml`
  No Changes to `/global/u2/j/jaimerz/PhD/MicroCanonicalHMC.jl/examples/Manifest.toml`


In [18]:
using Plots, ForwardDiff, LinearAlgebra, StaticArrays

In [19]:
function symplectic_integrate(x₀, p₀, U; N=50, ϵ=0.1, micro=false)
    
    V(p) = (micro ? d*log(dot(p,p)/d)/2 : (dot(p,p))/2)
    H(x,p) = U(x) - V(p)

    δUδx = x -> ForwardDiff.gradient(U, x)
    δVδp = p -> ForwardDiff.gradient(V, p)

    xᵢ, pᵢ = x₀, p₀
    d = length(x₀)
    δUδxᵢ = δUδx(xᵢ)
    
    history = []

    for i=1:N
        xᵢ₊₁    = xᵢ - ϵ * (δVδp(pᵢ) - ϵ/2 * δUδxᵢ)
        δUδxᵢ₊₁ = δUδx(xᵢ₊₁)
        pᵢ₊₁    = pᵢ - ϵ/2 * (δUδxᵢ₊₁ + δUδxᵢ)
        xᵢ, pᵢ, δUδxᵢ = xᵢ₊₁, pᵢ₊₁, δUδxᵢ₊₁
        Hᵢ = H(xᵢ,pᵢ)
        push!(history, (xᵢ, Hᵢ))
    end

    return history
end

symplectic_integrate (generic function with 1 method)

In [20]:
U((x,y),) = -max(0,(((1.0 - x)^2 + 100.0 * (y - x^2)^2) + 10(x^2 + y^2)))^(1/2)

U (generic function with 1 method)

In [21]:
x = range(-4, 4, length=300)
y = range(-2, 4, length=300);

In [22]:
history1 = symplectic_integrate(@SVector[1.3,2], @SVector[0.1, 0.1], I, U, N=200000, ϵ=0.0001, micro=false);

LoadError: MethodError: no method matching symplectic_integrate(::SVector{2, Float64}, ::SVector{2, Float64}, ::UniformScaling{Bool}, ::typeof(U); N::Int64, ϵ::Float64, micro::Bool)

[0mClosest candidates are:
[0m  symplectic_integrate(::Any, ::Any, ::Any; N, ϵ, micro)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[19]:1[24m[39m


In [23]:
history2 = symplectic_integrate(@SVector[1.3,2], @SVector[0.1, 0.1], I, U, N=50000000, ϵ=0.000001, micro=true);

LoadError: MethodError: no method matching symplectic_integrate(::SVector{2, Float64}, ::SVector{2, Float64}, ::UniformScaling{Bool}, ::typeof(U); N::Int64, ϵ::Float64, micro::Bool)

[0mClosest candidates are:
[0m  symplectic_integrate(::Any, ::Any, ::Any; N, ϵ, micro)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[19]:1[24m[39m


In [24]:
anim = Plots.@animate for h in first.(history1)[1:300:end]
    contourf(x, y, U.(tuple.(x',y)), clims=(-14,0), cmap=:acton)
    scatter!([h[1]], [h[2]], ylim=(-2,4), xlim=(-3,3), c=:black, ms=8, label=false)
end fps=5
mp4(anim, "hmc.mp4")

LoadError: UndefVarError: `history1` not defined

In [25]:
anim = Plots.@animate for h in first.(history2)[1:100000:end]
    contourf(x, y, U.(tuple.(x',y)), clims=(-14,0), cmap=:acton)
    scatter!([h[1]], [h[2]], ylim=(-2,4), xlim=(-3,3), c=:black, ms=8, label=false)
end fps=5
mp4(anim, "mchmc.mp4")

LoadError: UndefVarError: `history2` not defined